In [3]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [5]:
class Model1Xgb:
    def __init__(self):
        self.model = None
    def fit(self, tr_x, tr_y, va_x, va_y):
        params ={
            'objective':'binary:logistic',
            'verbosity':0, 
            'random_state':71, 
            'eval_metrics':'logloss'
        }
        num_rounds=10 
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label= va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_rounds, 
                              evals=watchlist)
    def predict(self, x):
        data = xgb.DMatrix(x)
        pred= self.model.predict(data)
        return pred

In [6]:
class Model1NN:
    def __init__(self):
        self.model = None
        self.scaler= None
    def fit(self, tr_x, tr_y, va_x, va_y):
        
        batch_size= 128
        epochs=10
        
        self.scaler = StandardScaler()
        tr_x =self.scaler.fit_transform(tr_x)
        va_x =self.scaler.transform(va_x)
        
        model = Sequential()
        model.add(Dense(256, activation='relu', input_shape=(tr_x.shape[1],)))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.compile(loss='binary_crossentropy', optimizer='adam')
        
        history= model.fit(tr_x, tr_y, batch_size= batch_size, 
                           epochs=epochs, verbose=1, validation_data=(va_x, va_y))
        self.model =model 
    def predict(self, x):
        x=self.scaler.transform(x)
        pred=self.model.predict_proba(x).reshape(-1)
        return pred

In [7]:
class Model2Linear:
    def __init__(self):
        self.model =None
        self.scaler= None
    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler= StandardScaler()
        tr_x = self.scaler.fit_transform(tr_x)
        va_x = self.scaler.transform(va_x) # 없는게 맞는지 의문
        
        self.model = LogisticRegression(solver='lbfgs', C=1.0)
        self.model.fit(tr_x, tr_y)
    def predict(self, x):
        x=self.scaler.transform(x)
        pred= self.model.predict_proba(x)[:,1]
        return pred